<b><h1 style="color:blue;">COURSERA IBM DATA SCIENCCE - CAPSTONE</h1></b>

This notebook is for the IBM Data Science Capstone Project on Coursera
______________________________________________________________________

In [1]:
import numpy as np
import pandas as pd

In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


<b>1. Webscrapping - Torronto Neighborhoods Data</b>

In [3]:
from bs4 import BeautifulSoup # this module helps in web scrapping.
import requests  # this module helps us to download a web page

In [4]:
#get url and create bs object
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
data = requests.get(url).text
nbhoods_bs = BeautifulSoup(data,"html.parser")  

<details><summary>Tips</summary>

```**Tips for  Webscraping Updated Table in Week3 Peer Graded Assignment

**After retreiving the URL and creating a Beautiful soup object** 

**Firstly create a list**  

**Later after finding the table and table data  create a dictionary called cell having 3 keys PostalCode, Borough and Neighborhood.**

**As postal code contains upto 3 characters extract that using tablerow.p.text**

**Next use split ,strip and replace functions for getting Borough and Neighborhood information.**.

**Append to the list**  

**Create a dataframe with list**
```

</details>



In [5]:
table_contents = []
toronto_table = nbhoods_bs.find('table')
for row in toronto_table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['Postal Code'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
nbhoods_df = pd.DataFrame(table_contents)
nbhoods_df['Borough'] = nbhoods_df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

nbhoods_df.tail()


,Postal Code,Borough,Neighborhood
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."
102,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [6]:
nbhoods_df.shape

(103, 3)

<b>2. Use Geocoder to get coordinates for each borough</b>

In [7]:
#!pip install geocoder
import geocoder


<details><summary>Doesnt work</summary>

```
</details># initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
postal_code = nbhoods_df['PostalCode']

while(lat_lng_coords is None):
    g = geocoder.google('{}, Toronto, Ontario'.format('postal_code'))
    lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

nbhoods_df['Latitude'] = latitude
nbhoods_df['Longitude'] = longitude

nbhoods_df.head(10)
```

In [8]:
#import coord file
coord = pd.read_csv('Geospatial_Coordinates.csv')
coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [9]:
nbhoods_df = nbhoods_df.merge(coord, on=['Postal Code'])
nbhoods_df

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto Business,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
